In [ ]:
import tensorflow
import pygad.kerasga
import numpy
import pygad
import os
from sklearn.model_selection import train_test_split

# Base Methods

In [ ]:
def fitness_func(solution, sol_idx):
    global train_data, train_labels, keras_ga, model
    predictions = pygad.kerasga.predict(model=model, solution=solution, data=train_data)

    ce = tensorflow.keras.losses.BinaryCrossentropy()
    solution_fitness = 1.0 / (ce(train_labels, predictions).numpy() + 0.00000001)

    return solution_fitness

In [ ]:
def callback_generation(ga_instance):
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))

In [ ]:
def dir_train_test_split(split, directory):
    file_paths = []
    for fileName in os.listdir(directory):
        file_paths.append(os.path.join(directory, fileName))
    data = numpy.array(file_paths)
    return train_test_split(data,test_size=split) 

In [ ]:
def load_images(file_paths):
    data = []
    for filePath in list(file_paths):
        img = tensorflow.keras.preprocessing.image.load_img(filePath, color_mode = "grayscale")
        img_array = tensorflow.keras.preprocessing.image.img_to_array(img)
        normalized_img = tensorflow.image.resize_with_crop_or_pad(img_array, 256, 256)
        data.append(normalized_img)
    return numpy.array(data)

In [ ]:
def load_labels(file_paths):
    labels = []
    for path in file_paths:
        if "parkinson" in path:
            labels.append(1)
        else:
            labels.append(0)
    return tensorflow.keras.utils.to_categorical(numpy.array(labels))

In [ ]:
def build_keras_model():
    model = tensorflow.keras.Sequential([
        tensorflow.keras.layers.InputLayer(input_shape=(256, 256, 1)),
        tensorflow.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tensorflow.nn.relu),
        tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tensorflow.keras.layers.Flatten(),
        tensorflow.keras.layers.Dense(2, activation="softmax")
    ])
    return model

In [ ]:
def load_data():
    parkinson_test, parkinson_train = dir_train_test_split(0.8, "/Users/timo/Documents/code/fhnw/spiral-ml/spiral_drawings/parkinson")
    healthy_test, healthy_train = dir_train_test_split(0.8, "/Users/timo/Documents/code/fhnw/spiral-ml/spiral_drawings/healthy")

    train = numpy.concatenate((parkinson_train, healthy_train))
    test = numpy.concatenate((parkinson_test, healthy_test))

    train_data = load_images(train)
    test_data = load_images(test)
    
    train_labels = load_labels(train)
    test_labels = load_labels(test)
    return train_data, train_labels, test_data, test_labels

# Load Data

In [ ]:
train_data, train_labels, test_data, test_labels = load_data()

In [ ]:
len(train_data)+len(test_data)

# Base Image Classification

In [ ]:
simple_model = build_keras_model()
simple_model.compile(optimizer='adam', loss=tensorflow.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

simple_model.fit(
    train_data,
    train_labels,
    validation_split=0.1,
    epochs=10
)

In [ ]:
_, baseline_model_accuracy = simple_model.evaluate(test_data, test_labels, verbose=0)
baseline_model_accuracy

# Optimize CNN with genetic algorithm

In [ ]:
model = build_keras_model()

keras_ga = pygad.kerasga.KerasGA(model=model,num_solutions=10)

initial_population = keras_ga.population_weights 

ga_instance = pygad.GA(num_generations=200,
                       num_parents_mating=3,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       on_generation=callback_generation,
                       mutation_probability=0.40)

ga_instance.run()

Plot the evolution

In [ ]:
plot = ga_instance.plot_fitness(title="PyGAD & Keras - Iteration vs. Fitness", linewidth=4)

Show the best solution

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

Evaluate best solution

In [ ]:
# Make predictions based on the best solution.
predictions = pygad.kerasga.predict(model=model,solution=solution,data=test_data)


# Calculate the categorical crossentropy for the trained model.
cce = tensorflow.keras.losses.BinaryCrossentropy()
print("Categorical Crossentropy : ", cce(test_labels, predictions).numpy())

# Calculate the classification accuracy for the trained model.
ca = tensorflow.keras.metrics.CategoricalAccuracy()
ca.update_state(test_labels, predictions)
accuracy = ca.result().numpy()
print("Accuracy : ", accuracy)